______
<img style="float: right;" src="./images/headerlogo.png">
## Google Earth Engine Tutorial
### Image Classification
_____

Image classification means mapping the values captured by remote sensors that are encoded as image digital levels to specific land cover types. <br>
Classifying remotely sensed data into a thematic map is a relevant task beacuse the resulting information is the basis for many environmental and socioeconomic applications. In this example, a crop map type is produced by applying a [*supervised*](https://en.wikipedia.org/wiki/Supervised_learning) [*Random Forest*](https://en.wikipedia.org/wiki/Random_forest) algorithm. 

#### Supervised Classification

A supervised classification method consists of training a classifier (algorithm) using (ground) truth data to classify  subsequently unseen data. <br>
The process flow in a supervised classification includes basically, the following steps:
* Collect the ground data. This data is used to train the classifier and validate its results.
* Train the classifier.
* Apply the classifier to produce a classified image.
* Assess the classification accuracy.

In [6]:
import ee
from IPython.display import Image
ee.Initialize()
# Load the image
image= ee.Image('users/rosamaguilar/tutorial/subset')
Image(url=image.getThumbUrl({'min': 0, 'max': 2048,'bands': 'b5,b3,b2'}))

In [7]:
# Two fusion tables previously created will be used for training set and a testing set. These tables contain ground truth data
# Load tables into feature collection
TR = ee.FeatureCollection('ft:11shsjBBaQQq-gTWd4M3QGl32GF47_eT5GVcQaKYO')
TS = ee.FeatureCollection('ft:1O8qBNitB6Z3iyrX5-B0mA2I1HnH05Qw8cDmsbaNj')

In [8]:
# get feature names to use in classification
feats = TR.first().propertyNames()
# remove AA_Class that is the label, system:index that is an internal attribute and myid that is not relevant for classification
feats = feats.removeAll(['AA_Class','myid','system:index'])  
print(feats.getInfo())

['b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b1']


In [9]:
import pprint
# define an train  Random Forest clasifier
rf = ee.Classifier.randomForest(10).train(TR, "AA_Class", feats) 
# apply the classifier to the training test for accuracy assessment
result = TS.classify(rf, "rfclass")
# Calculate the confusion matrix
error = result.errorMatrix('AA_Class', 'rfclass')
errMatrix = error.array().slice(0, 1, 6).slice(1, 1, 6)
print("Confusion Matrix for testing set")
pprint.pprint(errMatrix.getInfo())
# Extract Overall accuracy and kappa coefficient
rf_oa_ts = result.errorMatrix('AA_Class', 'rfclass').accuracy().getInfo()
rf_kappa_ts = result.errorMatrix('AA_Class', 'rfclass').kappa().getInfo()
print("RF OA: ",rf_oa_ts, "RF kappa: ",rf_kappa_ts )

Confusion Matrix for testing set
[[108, 34, 6, 11, 15],
 [31, 134, 16, 34, 9],
 [24, 22, 47, 16, 4],
 [19, 38, 14, 120, 26],
 [14, 31, 4, 26, 130]]
RF OA:  0.5777063236870311 RF kappa:  0.46405811934410174


In [10]:
# classify the image and display result
classified = image.classify(rf, "rfclass")
clas_col = ','.join(['000000','bdc3c7','1e8449','d68910','2ecc71','f9e79f'])   
Image(url = classified.getThumbUrl({'min': 0, 'max': 5, 'palette':clas_col}))


Classification result could be improved by adding more data such as vegetation indices and textural features. You may try to perform a new classification considering those features.